[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/siowcm/capstone/blob/main/code/tf_v03_ssd_resnet50_v1_fpn_640x640.ipynb)


# Capstone Project: Mask Wearing Detector
---


## Table of Contents

1. Preprocessing
2. Modelling
---

In [1]:
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/libcudnn8_8.1.0.77-1+cuda11.2_amd64.deb
!dpkg -i libcudnn8_8.1.0.77-1+cuda11.2_amd64.deb
!ls -l /usr/lib/x86_64-linux-gnu/libcudnn.so.8*

--2022-02-16 05:38:07--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/libcudnn8_8.1.0.77-1+cuda11.2_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.199.20.126
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.199.20.126|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 430460776 (411M) [application/x-deb]
Saving to: ‘libcudnn8_8.1.0.77-1+cuda11.2_amd64.deb’

libcudnn8_8.1.0.77- 100%[===================>] 410.52M   203MB/s    in 2.0s    

2022-02-16 05:38:10 (203 MB/s) - ‘libcudnn8_8.1.0.77-1+cuda11.2_amd64.deb’ saved [430460776/430460776]

(Reading database ... 155113 files and directories currently installed.)
Preparing to unpack libcudnn8_8.1.0.77-1+cuda11.2_amd64.deb ...
Unpacking libcudnn8 (8.1.0.77-1+cuda11.2) over (8.0.5.39-1+cuda11.1) ...
Setting up libcudnn8 (8.1.0.77-1+cuda11.2) ...
lrwxrwxrwx 1 root root     17 Jan 25  2021 /usr/lib/x86_64-linux-gnu/lib

In [2]:
!nvidia-smi

Wed Feb 16 05:39:12 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install -U opencv-python-headless==4.5.2.52
#!pip install "opencv-python-headless<4.3"

     |████████████████████████████████| 38.2 MB 1.2 MB/s 


In [4]:
import os, shutil
# import cv2
import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [5]:
os.mkdir("workspace")
os.mkdir("workspace/training")
os.mkdir("workspace/training/annotations")
os.mkdir("workspace/training/exported-models")
os.mkdir("workspace/training/images")
os.mkdir("workspace/training/images/train")
os.mkdir("workspace/training/images/test")
os.mkdir("workspace/training/models")
os.mkdir("workspace/training/pre-trained-models/")

In [6]:
#import data
!git clone https://github.com/siowcm/capstone.git
!git clone https://github.com/tensorflow/models.git
!git clone https://github.com/cocodataset/cocoapi.git

Cloning into 'capstone'...
remote: Enumerating objects: 5881, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 5881 (delta 49), reused 70 (delta 28), pack-reused 5790
Receiving objects: 100% (5881/5881), 418.57 MiB | 24.50 MiB/s, done.
Resolving deltas: 100% (897/897), done.
Checking out files: 100% (5792/5792), done.
Cloning into 'models'...
remote: Enumerating objects: 69147, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 69147 (delta 2), reused 16 (delta 1), pack-reused 69128
Receiving objects: 100% (69147/69147), 577.29 MiB | 27.10 MiB/s, done.
Resolving deltas: 100% (48703/48703), done.
Cloning into 'cocoapi'...
remote: Enumerating objects: 975, done.
remote: Total 975 (delta 0), reused 0 (delta 0), pack-reused 975
Receiving objects: 100% (975/975), 11.72 MiB | 25.93 MiB/s, done.
Resolving deltas: 100% (576/576), done.


In [7]:
#protobuf installation
%%bash
cd /content/models/research
pwd
protoc object_detection/protos/*.proto --python_out=.

/content/models/research


In [8]:
#COCO API installation
%%bash
cd cocoapi/PythonAPI
pwd
make
cp -r pycocotools /content/models/research

/content/cocoapi/PythonAPI
python setup.py build_ext --inplace
running build_ext
cythoning pycocotools/_mask.pyx to pycocotools/_mask.c
building 'pycocotools._mask' extension
creating build
creating build/common
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/pycocotools
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-pX47U3/python3.7-3.7.12=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-pX47U3/python3.7-3.7.12=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.7/dist-packages/numpy/core/include -I../common -I/usr/include/python3.7m -c ../common/maskApi.c -o build/temp.linux-x86_64-3.7/../common/maskApi.o -Wno-cpp -Wno-unused-function -std=c99
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-m

../common/maskApi.c: In function ‘rleDecode’:
../common/maskApi.c:46:7: warning: this ‘for’ clause does not guard... [-Wmisleading-indentation]
       for( k=0; k<R[i].cnts[j]; k++ ) *(M++)=v; v=!v; }}
       ^~~
../common/maskApi.c:46:49: note: ...this statement, but the latter is misleadingly indented as if it were guarded by the ‘for’
       for( k=0; k<R[i].cnts[j]; k++ ) *(M++)=v; v=!v; }}
                                                 ^
../common/maskApi.c: In function ‘rleFrPoly’:
../common/maskApi.c:166:3: warning: this ‘for’ clause does not guard... [-Wmisleading-indentation]
   for(j=0; j<k; j++) x[j]=(int)(scale*xy[j*2+0]+.5); x[k]=x[0];
   ^~~
../common/maskApi.c:166:54: note: ...this statement, but the latter is misleadingly indented as if it were guarded by the ‘for’
   for(j=0; j<k; j++) x[j]=(int)(scale*xy[j*2+0]+.5); x[k]=x[0];
                                                      ^
../common/maskApi.c:167:3: warning: this ‘for’ clause does not guard... [-Wmisleading

In [9]:
#object detection API installation
%%bash
cd /content/models/research
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1686356 sha256=e87d9de13136e70ed16dfa9891be118b31d0bedbcf21b5c2d4f3603804c4d82d
  Stored in directory: /tmp/pip-ephem-wheel-cache-9zp1s5kk/wheels/fa/a4/d2/e9a5057e414fd46c8e543d2706cd836d64e1fcd9eccceb2329
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-py3-none-any.whl size=22257 sha256=c68dbf72d78d6d0fbbc99a411f187afd3e071c946de8742ae7a25bbc88720664
  Stored in directory: /root/.cache/pip/wheels/d2/f1/1f/041add21dc9c4220157f1bd2bd6afe1f1a49524c3396b94401
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78544 sha256=dacbb24775e8493faf1935ad6682c3dfb049871af331c3f48b6390ffd2ae3fcf
  Stored in directory: /root/.cache/pip/wheels/a4/61/fd/c57e374e580aa78a45ed78d5859b3a44436af17e22ca53284f
  Created wheel for avro-python3: filename=avro_python3-1.10.2-py3-none-any.whl size=44010 sha256=6d83a11c1ca177155b074e31a6f6e7d03f8458359d44f1c180

  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
multiprocess 0.70.12.2 requires dill>=0.3.4, but you have dill 0.3.1.1 which is incompatible.
gym 0.17.3 requires cloudpickle<1.7.0,>=1.2.0, but you have cloudpickle 2.0.0 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [10]:
#transfer updated setup.py
shutil.copy(
    "capstone/code/setup.py",
    "models/research"
)

'models/research/setup.py'

In [11]:
#object detection API installation
%%bash
cd /content/models/research
python -m pip install .

Processing /content/models/research
INFO: pip is looking at multiple versions of tensorflow-text to determine which version is compatible with other requirements. This could take a while.
  Using cached tensorflow_text-2.8.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (4.9 MB)
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1686373 sha256=f791fa08973beb508aa4eb360846e977478fe976bebc79425590b39e66352e3c
  Stored in directory: /tmp/pip-ephem-wheel-cache-jedwuu1j/wheels/fa/a4/d2/e9a5057e414fd46c8e543d2706cd836d64e1fcd9eccceb2329
Successfully built object-detection
  Attempting uninstall: tensorflow-io-gcs-filesystem
    Found existing installation: tensorflow-io-gcs-filesystem 0.24.0
    Uninstalling tensorflow-io-gcs-filesystem-0.24.0:
      Successfully uninstalled tensorflow-io-gcs-filesystem-0.24.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-

  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.


In [12]:
#Test installation
#!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

In [13]:
# define path
IMGS_PATH = "capstone/input_data/images/"
ANNOT_PATH = "capstone/input_data/annotations/"
CROPPED_PATH = "capstone/output_data/cropped_images/"
CSV_PATH = "capstone/output_data/csv/"

In [14]:
df = pd.read_csv(CSV_PATH + "mask.csv")

In [15]:
# manually splitting the the data ensuring classes are evenly distributed and
# no data leakage

# saving the list of file names according to their classes
incorrect = list(
    set(df[df.label == "mask_weared_incorrect"]["images"].to_list())
)

without = list(set(df[df.label == "without_mask"]["images"].to_list()))

with_mask = list(set(df[df.label == "with_mask"]["images"].to_list()))

# ensuring that file names does not appear on other list
incorrect_u = incorrect
without_u = [i for i in without if i not in incorrect]
with_mask_U = [i for i in with_mask if i not in incorrect and i not in without]

# split data into train and test
test_incorrect = incorrect_u[int(0.8 * len(incorrect_u)) :]
train_incorrect = incorrect_u[: int(0.8 * len(incorrect_u))]

test_without = without_u[int(0.8 * len(without_u)) :]
train_without = without_u[: int(0.8 * len(without_u))]

test_with_mask = with_mask_U[int(0.8 * len(with_mask_U)) :]
train_with_mask = with_mask_U[: int(0.8 * len(with_mask_U))]

training_set = train_incorrect + train_without + train_with_mask
testing_set = test_incorrect + test_without + test_with_mask

In [16]:
# copy files into train and test folder
for file in training_set:
    source = IMGS_PATH + file
    destination = "workspace/training/images/train/"
    shutil.copy(source, destination)

for file in [i.replace(".png", ".xml") for i in training_set]:
    source = ANNOT_PATH + file
    destination = "workspace/training/images/train/"
    shutil.copy(source, destination)


for file in testing_set:
    source = IMGS_PATH + file
    destination = "workspace/training/images/test/"
    shutil.copy(source, destination)

for file in [i.replace(".png", ".xml") for i in testing_set]:
    source = ANNOT_PATH + file
    destination = "workspace/training/images/test/"
    shutil.copy(source, destination)

In [17]:
# generate label map
labels = [
    {"name": "with_mask", "id": 1},
    {"name": "without_mask", "id": 2},
    {"name": "mask_weared_incorrect", "id": 3},
]

with open("workspace/training/annotations/" + "label_map.pbtxt", "w") as f:
    for label in labels:
        f.write("item { \n")
        f.write("\tname:'{}'\n".format(label["name"]))
        f.write("\tid:{}\n".format(label["id"]))
        f.write("}\n")

In [18]:
'''
troubleshooting code. Transfer 1 set of img/ annotation to train & test instead
'''

# shutil.copy(
#     "capstone/input_data/annotations/maksssksksss0.xml",
#     "workspace/training/images/train/",
# )
# shutil.copy(
#     "capstone/input_data/images/maksssksksss0.png",
#     "workspace/training/images/train/",
# )

# shutil.copy(
#     "capstone/input_data/annotations/maksssksksss0.xml",
#     "workspace/training/images/test/",
# )
# shutil.copy(
#     "capstone/input_data/images/maksssksksss0.png",
#     "workspace/training/images/test/",
# )

'\ntroubleshooting code. Transfer 1 set of img/ annotation to train & test instead\n'

In [19]:
#download script to generate tfrecord
os.mkdir("scripts")
os.mkdir("scripts/preprocessing")
!wget https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/_downloads/da4babe668a8afb093cc7776d7e630f3/generate_tfrecord.py -P scripts/preprocessing

--2022-02-16 05:42:25--  https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/_downloads/da4babe668a8afb093cc7776d7e630f3/generate_tfrecord.py
Resolving tensorflow-object-detection-api-tutorial.readthedocs.io (tensorflow-object-detection-api-tutorial.readthedocs.io)... 104.17.32.82, 104.17.33.82, 2606:4700::6811:2052, ...
Connecting to tensorflow-object-detection-api-tutorial.readthedocs.io (tensorflow-object-detection-api-tutorial.readthedocs.io)|104.17.32.82|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6410 (6.3K) [text/x-python]
Saving to: ‘scripts/preprocessing/generate_tfrecord.py’

generate_tfrecord.p 100%[===================>]   6.26K  --.-KB/s    in 0s      

2022-02-16 05:42:26 (35.4 MB/s) - ‘scripts/preprocessing/generate_tfrecord.py’ saved [6410/6410]



In [20]:
#create tfrecord
%%bash
cd scripts/preprocessing/
python generate_tfrecord.py -x ../../workspace/training/images/train -l ../../workspace/training/annotations/label_map.pbtxt -o ../../workspace/training/annotations/train.record
python generate_tfrecord.py -x ../../workspace/training/images/test -l ../../workspace/training/annotations/label_map.pbtxt -o ../../workspace/training/annotations/test.record  

Successfully created the TFRecord file: ../../workspace/training/annotations/train.record
Successfully created the TFRecord file: ../../workspace/training/annotations/test.record


In [21]:
#download pre-trained model
%%bash
cd workspace/training/pre-trained-models/
wget -nv http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz 
tar -xf ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz

2022-02-16 05:42:44 URL:http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz [244817203/244817203] -> "ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz" [1]


In [22]:
'''
manually update pipeline.config or copy from github
=========
3 num_classes: 3
131 batch_size: 8
161 fine_tune_checkpoint: "workspace/training/pre-trained-models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/checkpoint" # Path to checkpoint of pre-trained model
172 182 label_map_path: "workspace/training/annotations/label_map.pbtxt" # Path to label map file
174 186 input_path: "workspace/training/annotations/train.record" # Path to training TFRecord file
'''

#copy pipline config

os.mkdir("workspace/training/models/my_ssd_resnet50_v1_fpn/")
shutil.copy(
    "capstone/code/pipeline.config",
    "workspace/training/models/my_ssd_resnet50_v1_fpn/"
)


'workspace/training/models/my_ssd_resnet50_v1_fpn/pipeline.config'

In [23]:
shutil.copy(
    "models/research/object_detection/model_main_tf2.py",
    "workspace/training/",
)

'workspace/training/model_main_tf2.py'

In [28]:
import time
start = time.time()

In [29]:
#start training
%%bash
cd workspace/training
python model_main_tf2.py --model_dir=models/my_ssd_resnet50_v1_fpn --pipeline_config_path=models/my_ssd_resnet50_v1_fpn/pipeline.config

2022-02-16 05:47:01.122199: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0216 05:47:01.125846 139721902290816 mirrored_strategy.py:376] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: None
I0216 05:47:01.130988 139721902290816 config_util.py:552] Maybe overwriting train_steps: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0216 05:47:01.131326 139721902290816 config_util.py:552] Maybe overwriting use_bfloat16: False
Instructions for updating:
rename to distribute_datasets_from_function
W0216 05:47:01.165904 139721902290816 deprecation.py:347] From /usr/local/lib/python3.7/dist-packages/object_detection/model_lib_v2.py:564: StrategyBase.experiment

In [30]:
end = time.time()
print(end - start)

1323.1783792972565


In [40]:
!python /content/models/research/object_detection/model_main_tf2.py \
--model_dir=/content/workspace/training/exported-models/my_model/checkpoint \
--pipeline_config_path=/content/workspace/training/exported-models/my_model/pipeline.config \
--checkpoint_dir=/content/workspace/training/models/my_ssd_resnet50_v1_fpn # The folder where the model has saved the checkpoints during training


W0216 07:22:45.766763 140338103375744 model_lib_v2.py:1090] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: None
I0216 07:22:45.767008 140338103375744 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0216 07:22:45.767144 140338103375744 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0216 07:22:45.767397 140338103375744 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W0216 07:22:45.767619 140338103375744 model_lib_v2.py:1111] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
2022-02-16 07:22:46.598776: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original confi